In [34]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re

In [3]:
url = "https://fbref.com/en/comps/13/3243/2019-2020-Ligue-1-Stats"

In [107]:
standard_stats = ["player","nationality","position","age","games","games_starts","minutes","minutes_90s","goals","assists","goals_pens","pens_made","pens_att","cards_yellow","cards_red","goals_per90","assists_per90","goals_assists_per90","goals_assists_pens_per90","xg","npxg","xa","npxg_xa","xg_per90","xa_per90","xg_xa_per90","npxg_per90","npxg_xa_per90"]

In [4]:
res = requests.get(url)

In [5]:
#to get over the scrap breaker
comm = re.compile("<!--|-->")
soup = BeautifulSoup(comm.sub("",res.text),'html.parser')

In [6]:
tables = soup.find_all("tbody")

In [10]:
def get_links(table):
    team_links = []
    teams = table.find_all("tr")
    for row in teams:
        squad = row.find_all("td",{'data-stat':'squad'})
        link = squad[0].find("a", href = True)
        team_links.append(link['href'])
    return team_links

In [11]:
df = []
df = get_links(tables[0])

In [12]:
df

['/en/squads/e2d8892c/2019-2020/Paris-Saint-Germain-Stats',
 '/en/squads/5725cc7b/2019-2020/Marseille-Stats',
 '/en/squads/b3072e00/2019-2020/Rennes-Stats',
 '/en/squads/cb188c0c/2019-2020/Lille-Stats',
 '/en/squads/132ebc33/2019-2020/Nice-Stats',
 '/en/squads/7fdd64e0/2019-2020/Reims-Stats',
 '/en/squads/d53c0b06/2019-2020/Lyon-Stats',
 '/en/squads/281b0e73/2019-2020/Montpellier-Stats',
 '/en/squads/fd6114db/2019-2020/Monaco-Stats',
 '/en/squads/c0d3eab4/2019-2020/Strasbourg-Stats',
 '/en/squads/69236f98/2019-2020/Angers-Stats',
 '/en/squads/123f3efe/2019-2020/Bordeaux-Stats',
 '/en/squads/d7a486cd/2019-2020/Nantes-Stats',
 '/en/squads/fb08dbb3/2019-2020/Brest-Stats',
 '/en/squads/f83960ae/2019-2020/Metz-Stats',
 '/en/squads/8dfb7350/2019-2020/Dijon-Stats',
 '/en/squads/d298ef2c/2019-2020/Saint-Etienne-Stats',
 '/en/squads/1cbf5f9e/2019-2020/Nimes-Stats',
 '/en/squads/25622401/2019-2020/Amiens-Stats',
 '/en/squads/3f8c4b5f/2019-2020/Toulouse-Stats']

In [38]:
def modify_links(df_link):
    final_links = []
    for link in df_link:
        split = link.split("/")
        split[5] = split[5] + "-All-Competitions"
        split.append(split[5])
        split[5] = "all-comps"
        separator = "/"
        final_links.append("https://fbref.com"+separator.join(split))
    return final_links
        


In [42]:
df_again = modify_links(df)

# Scrapping Main

In [43]:
req = requests.get(df_again[0])

In [44]:
#to get over the scrap breaker
comm = re.compile("<!--|-->")
soup = BeautifulSoup(comm.sub("",req.text),'html.parser')

In [45]:
tables = soup.find_all("tbody")

In [46]:
len(tables)

13

In [51]:
players= tables[0].find_all("tr")

In [108]:
pre_df_player = dict()
for row in players:
        if(row.find('th',{"scope":"row"}) != None):
            
            
            for f in standard_stats:
                if f == "player":
                    cell = row.find("th",{"data-stat": f})
                else:                    
                    cell = row.find("td",{"data-stat": f})
                a = cell.text.strip().encode()
                text=a.decode("utf-8")
                if(text == ''):
                    text = '0'
                if((f!='player')&(f!='nationality')&(f!='position')&(f!='squad')&(f!='age')&(f!='birth_year')):
                    text = float(text.replace(',',''))
                if f in pre_df_player:
                    pre_df_player[f].append(text)
                else:
                    pre_df_player[f] = [text]
df_player = pd.DataFrame.from_dict(pre_df_player)
    

In [109]:
df_player

,player,nationality,position,age,games,games_starts,minutes,minutes_90s,goals,assists,...,goals_assists_pens_per90,xg,npxg,xa,npxg_xa,xg_per90,xa_per90,xg_xa_per90,npxg_per90,npxg_xa_per90
0,Ángel Di María,ar ARG,"FW,MF",31,26.0,23.0,2000.0,22.2,8.0,14.0,...,0.95,9.1,7.6,13.9,21.5,0.41,0.63,1.04,0.34,0.97
1,Keylor Navas,cr CRC,GK,32,21.0,21.0,1890.0,21.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00
2,Thiago Silva,br BRA,DF,34,21.0,20.0,1571.0,17.5,0.0,0.0,...,0.00,0.2,0.2,0.0,0.2,0.01,0.00,0.01,0.01,0.01
3,Idrissa Gana Gueye,sn SEN,MF,29,20.0,19.0,1653.0,18.4,1.0,2.0,...,0.16,1.2,1.2,3.0,4.2,0.06,0.16,0.23,0.06,0.23
4,Kylian Mbappé,fr FRA,FW,20,20.0,17.0,1513.0,16.8,18.0,5.0,...,1.37,17.1,17.1,8.5,25.6,1.01,0.51,1.52,1.01,1.52
5,Juan Bernat,es ESP,"DF,MF",26,18.0,16.0,1388.0,15.4,0.0,4.0,...,0.26,0.5,0.5,2.3,2.8,0.03,0.15,0.18,0.03,0.18
6,Marco Verratti,it ITA,MF,26,20.0,15.0,1482.0,16.5,0.0,5.0,...,0.30,0.2,0.2,2.9,3.2,0.01,0.18,0.19,0.01,0.19
7,Marquinhos,br BRA,"MF,DF",25,19.0,15.0,1476.0,16.4,3.0,1.0,...,0.24,1.8,1.8,0.5,2.3,0.11,0.03,0.14,0.11,0.14
8,Neymar,br BRA,"MF,FW",27,15.0,15.0,1318.0,14.6,13.0,6.0,...,1.02,12.7,8.9,5.9,14.7,0.87,0.40,1.27,0.61,1.01
9,Mauro Icardi,ar ARG,FW,26,20.0,14.0,1267.0,14.1,12.0,2.0,...,0.99,9.9,9.9,2.6,12.5,0.70,0.18,0.89,0.70,0.89
